## 파일이 너무 크면 pandas로 읽어 지지 않음
## 모두 csv로 바꾼 후 laod
## 컬럼 많은 파일 합치기

## 하지만 카테고리 라벨이 없어서 로컬 LLM (Gemma3)를 사용하여 카테고리를 라벨링 하기로 결정!!

### DB 해야할 거
1. 음식 테이블의 스키마 정하기
2. 카테고리 LLM으로 채워넣기 (not null)
3. table 관계 짜서 mysql에 넣기
4. 입출력 코드 일반화
5. 서버와 연결

In [1]:
import pandas as pd
import os

In [50]:
# 왜 안됨?
columns_types = {
    '식품코드': 'object',
    '식품명': 'object',
    '데이터구분코드': 'object',
    '데이터구분명': 'object',
    '식품기원코드': 'object',
    '식품기원명': 'object',
    '식품대분류코드': 'object',
    '식품대분류명': 'object',
    '대표식품코드': 'object',
    '대표식품명': 'object',
    '식품중분류코드': 'object',
    '식품중분류명': 'object',
    '식품소분류코드': 'object',
    '식품소분류명': 'object',
    '식품세분류코드': 'object',
    '식품세분류명': 'object',
    '출처코드': 'int32',
    '출처명': 'object',
    '품목제조보고번호': 'object',
    '업체명': 'object',
    '제조사명': 'object',
    '수입업체명': 'object',
    '유통업체명': 'object',
    '수입여부': 'object',
    '원산지국코드': 'object',
    '원산지국명': 'object',
    '데이터생성방법코드': 'int32',
    '데이터생성방법명': 'object',
    '데이터기준일자': 'object',
    '제공기관코드': 'int32',
    '제공기관명': 'object',
    '폐기율(%)': 'float64',
    '식품중량': 'object',
    '1회 섭취참고량': 'object',
    '영양성분함량기준량': 'object',
    '에너지(kcal)': 'float64',
    '수분(g)': 'float64',
    '단백질(g)': 'float64',
    '지방(g)': 'float64',
    '회분(g)': 'float64',
    '탄수화물(g)': 'float64',
    '당류(g)': 'float64',
    '식이섬유(g)': 'float64',
    '칼슘(mg)': 'float64',
    '철(mg)': 'float64',
    '인(mg)': 'float64',
    '칼륨(mg)': 'float64',
    '나트륨(mg)': 'float64',
    '비타민A(μg RAE)': 'float64',
    '레티놀(μg)': 'float64',
    '베타카로틴(μg)': 'float64',
    '티아민(mg)': 'float64',
    '리보플라빈(mg)': 'float64',
    '니아신(mg)': 'float64',
    '비타민 C(mg)': 'float64',
    '비타민 D(μg)': 'float64',
    '콜레스테롤(mg)': 'float64',
    '포화지방산(g)': 'float64',
    '트랜스지방산(g)': 'float64'
}

In [16]:
all_columns_df = pd.read_csv("all_column.csv", encoding="utf-8-sig")

In [17]:
my_columns = all_columns_df.csv_name.dropna().to_list()

In [22]:
all_df = pd.DataFrame(columns=my_columns)
# all_df = all_df.astype(columns_types)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 59 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   식품코드          0 non-null      object
 1   식품명           0 non-null      object
 2   데이터구분코드       0 non-null      object
 3   데이터구분명        0 non-null      object
 4   식품기원코드        0 non-null      object
 5   식품기원명         0 non-null      object
 6   식품대분류코드       0 non-null      object
 7   식품대분류명        0 non-null      object
 8   대표식품코드        0 non-null      object
 9   대표식품명         0 non-null      object
 10  식품중분류코드       0 non-null      object
 11  식품중분류명        0 non-null      object
 12  식품소분류코드       0 non-null      object
 13  식품소분류명        0 non-null      object
 14  식품세분류코드       0 non-null      object
 15  식품세분류명        0 non-null      object
 16  출처코드          0 non-null      object
 17  출처명           0 non-null      object
 18  품목제조보고번호      0 non-null      object
 19  업체명           0 non-

In [23]:
file_dir = "source"

file_list = [table_file for table_file in os.listdir(file_dir) if table_file.endswith(".csv")]

In [24]:
for file in file_list:
    for chunk in pd.read_csv(str(os.path.join(file_dir, file)), chunksize=5000, encoding="utf-8-sig", dtype='object'):
      all_df = pd.concat([all_df, chunk.reindex(columns=my_columns)], ignore_index=True)

In [25]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613309 entries, 0 to 613308
Data columns (total 59 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   식품코드          613309 non-null  object
 1   식품명           613309 non-null  object
 2   데이터구분코드       613309 non-null  object
 3   데이터구분명        613309 non-null  object
 4   식품기원코드        397071 non-null  object
 5   식품기원명         397071 non-null  object
 6   식품대분류코드       397071 non-null  object
 7   식품대분류명        397071 non-null  object
 8   대표식품코드        397071 non-null  object
 9   대표식품명         397071 non-null  object
 10  식품중분류코드       397071 non-null  object
 11  식품중분류명        397071 non-null  object
 12  식품소분류코드       397071 non-null  object
 13  식품소분류명        397071 non-null  object
 14  식품세분류코드       397071 non-null  object
 15  식품세분류명        397071 non-null  object
 16  출처코드          613309 non-null  object
 17  출처명           613309 non-null  object
 18  품목제조보고번호      494246 non

In [26]:
unique_fid_all_df = all_df.groupby('식품코드').first()

In [54]:
unique_fid_all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166248 entries, D101-004160000-0001 to R320-025030002-0000
Data columns (total 58 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   식품명           166248 non-null  object
 1   데이터구분코드       166248 non-null  object
 2   데이터구분명        166248 non-null  object
 3   식품기원코드        166248 non-null  object
 4   식품기원명         166248 non-null  object
 5   식품대분류코드       166248 non-null  object
 6   식품대분류명        166248 non-null  object
 7   대표식품코드        166248 non-null  object
 8   대표식품명         166248 non-null  object
 9   식품중분류코드       166248 non-null  object
 10  식품중분류명        166248 non-null  object
 11  식품소분류코드       166248 non-null  object
 12  식품소분류명        166248 non-null  object
 13  식품세분류코드       166248 non-null  object
 14  식품세분류명        166248 non-null  object
 15  출처코드          166248 non-null  object
 16  출처명           166248 non-null  object
 17  품목제조보고번호      139968 non-null  object
 18

In [55]:
unique_fid_all_df.to_csv("combine_data.csv", encoding="utf-8-sig")